## Smart Signatures with Atomic Swaps
## Exercises
#### 06.4.2b Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2022-01-22

* Sell ASA for a fixed price
* Combine Atomic Swap and Smart Signature
* Learn to use the If statement

## Setup
See notebook 04.1, the lines below will always automatically load functions in `algo_util.py`, the 5 accounts and the Purestake credentials

In [1]:
# Loading shared code and credentials
import sys, os

codepath = '..'+os.path.sep+'..'+os.path.sep+'sharedCode'
sys.path.append(codepath)
from algo_util import *
cred = load_credentials()

# Shortcuts to directly access the 3 main accounts
MyAlgo  = cred['MyAlgo']
Alice   = cred['Alice']
Bob     = cred['Bob']
Charlie = cred['Charlie']
Dina    = cred['Dina']

In [2]:
from algosdk import account, mnemonic
from algosdk.v2client import algod
from algosdk import transaction
from algosdk.transaction import PaymentTxn
from algosdk.transaction import AssetConfigTxn, AssetTransferTxn, AssetFreezeTxn
from algosdk.transaction import LogicSig, LogicSigTransaction

import algosdk.error
import json
import base64
import pandas as pd

In [3]:
from pyteal import *

In [4]:
# Initialize the algod client (Testnet or Mainnet)
algod_client = algod.AlgodClient(algod_token='', algod_address=cred['algod_test'], headers=cred['purestake_token'])
algod_client.status()["last-round"]

27780155

In [5]:
print(Alice['public'])
print(Bob['public'])
print(Charlie['public'])

GFJ3O3QBJ4H3KPXYA4MFTTDA7TMEPZXAEUHCSF3J6GEDAXMML4A55KYSL4
MB4W4II5EEZ7VA5RGS74BTND5CMYDXMUHGRJ42HQYOSMDJZ23ZPIUWRRKM
ZFTSGNBBFDH544IDUVB7S4B67CX6PJL6MMFJYT3GGDH2GWJ4N7H6VKRU2Y


## The Vending Machine – Exercises

#### Exercise 1: exchange rate (price)
* Change the smart contract so that 2 ALGOs are exchanged for 1 WSC coin. *Hint:* keep the decimals in mind!
* Test the smart contract. *Hint:* do not forget to adjust the amounts to reflect the new exchange rate.

#### Exercise 2: allow overpayment
* Change the smart contract so that it is OK if Bob pays too many ALGOs for a certain amount of WSC coins

#### Exercise 3: Anti money loundering
* Add an anti money-loundering condition
* Currently it is possible that Bob pays, but Charlie receives the coins
* We want to avoid that with a condition that whoever pays the ALGOs must the the one to receive the coins

**Hint:** In total you need to add 2 lines to the smart contract

#### Step 0: Get the status before the swap
* Also fund accounts if need be
* https://bank.testnet.algorand.network
* https://testnet.algoexplorer.io/dispenser

In [6]:
asset_holdings_df2(algod_client, MyAlgo['public'], Bob['public'], ["MyAlgo","Bob"])

,amountMyAlgo,unit,asset-id,name,decimals,amountBob
0,3.382550e+01,ALGO,0,Algorand,6,8.432000e+00
1,1.100000e+02,USDC,10458941,USDC,6,1.999975e+01
2,0.000000e+00,DRZY,12887013,Drizzy,1,NaN
3,9.000000e+02,HUSKEN,159159432,Husky Token,2,NaN
4,1.000000e+02,DWSC,159159534,Daniels WSC coin,2,NaN
5,9.969900e+09,Beer,159171974,Beer Coin,2,3.010002e+07
6,1.670000e+01,TEMP,159173248,Peters Tempcoin,1,8.330000e+01
7,1.000000e+02,TEMPYRY,159173545,Pyrys Tempcoin,1,NaN
8,9.170000e+01,TEMPYRY,159173586,Pyrys Tempcoin,1,8.300000e+00
9,9.900000e+02,WSC,159189398,Peters WSC coin,2,1.000000e+01


In [7]:
# Store the correct ID for the WSC coin
WSC_idx=159189398                         # <---------- Update!!

#### Step 1a: Write down the conditions as a PyTeal program
Small complication: there is no 

In [8]:
vending_condition = And (
    Global.group_size() == Int(2),                     # Vending is an atomic swap, hence 2 transactions
    Gtxn[0].type_enum() == TxnType.Payment,            # First TX is a payment TX ...
    Gtxn[0].xfer_asset() == Int(0),                    # ... in ALGOs
    Gtxn[1].type_enum() == TxnType.AssetTransfer,      # Second TX is an ASA transfer ...
    Gtxn[1].xfer_asset() == Int(WSC_idx),              # ... in the WSC coin
    Gtxn[0].amount() == Gtxn[1].asset_amount(),        # Exchange rate in SMALL units
    Gtxn[0].receiver() == Gtxn[1].sender(),            # Anti-theft
    Gtxn[0].rekey_to() == Global.zero_address(),
    Gtxn[0].close_remainder_to() == Global.zero_address(),
    Gtxn[1].rekey_to() == Global.zero_address(),
    Gtxn[1].asset_close_to() == Global.zero_address(),
)

optin_condition = And(
        Global.group_size() == Int(1),                # Opt-in is single transaction
        Txn.type_enum() ==TxnType.AssetTransfer,      # Opt-in is ASA transfer
        Txn.xfer_asset() == Int(WSC_idx),               # ... in the WSC coin
        Txn.asset_amount() == Int(0),                 # Payout impossible, opt-in is OK
        Txn.rekey_to() == Global.zero_address(),
        Txn.close_remainder_to() == Global.zero_address()
    )


import random
a = Int( random.randrange(2**32-1) )
random_cond = ( a == a )

fee_cond = Txn.fee() <= Int(1000)
    
vending_pyteal = And(
    random_cond, 
    fee_cond, 
    If(
        Global.group_size() == Int(1),        # condition
        optin_condition,                      # then-expression
        vending_condition                     # else-expression
        )
    )

#### Step 1b: Pyteal -> Teal

In [9]:
vending_teal = compileTeal(vending_pyteal, Mode.Signature, version=8)
#print(vending_teal)

#### Step 1c: Teal -> Bytecode for AVM

In [10]:
Vending = algod_client.compile(vending_teal)
Vending

{'hash': 'BGLPXQOVQYBR4HYI54XODRSAQPYM5KKLJFPZ4KRBVHAQ2F6QGYMJ7DWE74',
 'result': 'CCAFAZTjmtYKAASWk/RLIyMSMQGB6AcOEDIEIhJAAE0yBIECEjMAECISEDMAESQSEDMBECUSEDMBESEEEhAzAAgzARISEDMABzMBABIQMwAgMgMSEDMACTIDEhAzASAyAxIQMwEVMgMSEEIAIDIEIhIxECUSEDERIQQSEDESJBIQMSAyAxIQMQkyAxIQEEM='}

#### Step 2: Veding must opt-In
* Veding needs to be funded first (for TX fee and min holding)
* Veding then makes a 0 coin transaction to itself

##### Step 2a: Funding

In [11]:
# Step 2a.1: prepare transaction
sp = algod_client.suggested_params()

# How much? Min holdings + min holdings for 1 ASA + TX fee for a few several swaps
amt = int(0.1*1e6) + int(0.1*1e6) + int(20* 0.001*1e6)
txn = transaction.PaymentTxn(sender=MyAlgo['public'], sp=sp, 
                             receiver=Vending['hash'], amt=amt)

# Step 2a.(2+3+4): sign and send and wait ...
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27780331.
Waiting for round 27780331 to finish.
Waiting for round 27780332 to finish.
Transaction TFZDHVZHWEUN4KUFZ7ZWTADC7YD3LK2LRJN4ZPWXFVRMBS5YBFQA confirmed in round 27780333.


##### Step 2b: Opt-In

In [12]:
# Steo 2b.1: Prepare
sp = algod_client.suggested_params()
txn = AssetTransferTxn(Vending['hash'], sp, Vending['hash'], 0, WSC_idx)

# Steo 2b.2: Sign
encodedProg = Vending['result'].encode()
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn = LogicSigTransaction(txn, lsig)

# Step 2b.3 Send
txid = algod_client.send_transaction(stxn)

# Step 2b.4 Wait for ...
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27780332.
Waiting for round 27780332 to finish.
Waiting for round 27780333 to finish.
Waiting for round 27780334 to finish.
Transaction V4E6Z2S7VKNBGER46ZTG3MNTI33VCSYV3CYRUJHQIUMQE5EBVM3Q confirmed in round 27780335.


#### Step 3: MyAlgo puts 15 (full) WSC into the Vending machine

In [13]:
### Step 3.1: prepare and create TX

# Deal with SMALL units
WSC_decimals = algod_client.asset_info(WSC_idx)['params']['decimals']
amt = int( 15 * 10**WSC_decimals )             # <--------- 15 WSC coins in SMALL units

sp = algod_client.suggested_params()
txn = AssetTransferTxn(
    sender=MyAlgo['public'],
    sp=sp,
    receiver=Vending['hash'],               
    amt=amt,
    index=WSC_idx)                        # <-----  We are sending WSC

# Step 3.2 and 3.3: sign and send
stxn = txn.sign(MyAlgo['private'])
txid = algod_client.send_transaction(stxn)

# Step 3.4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

Current round is  27780335.
Waiting for round 27780335 to finish.
Waiting for round 27780336 to finish.
Transaction RJKHG4JP452GIBHZDXY36LGWUPWDHYTY6ZK6MJCL3X2BTYQ72UMQ confirmed in round 27780337.


## The vending machine is now ready
* We run several test cases
* These test cases have already been prepared and adapted to the exercise

#### Test 1: Bob buys
* Bob sends 0.2 ALGOS
* He asks for 0.1 WSC

In [ ]:
sp = algod_client.suggested_params()

# Step 1a: Prepare ALGO payment TXN from Bob to Veding
amt_1 = int(0.2*1e6)
txn_1 = PaymentTxn(Bob['public'], sp, Vending['hash'],amt_1)

# Step 1b: Prepare WSC transfer from Vending to Bob
WSC_decimals = algod_client.asset_info(WSC_idx)['params']['decimals']
amt_2 = int(0.1 * 10**WSC_decimals )
txn_2 = AssetTransferTxn(Vending['hash'], sp, Bob['public'], amt_2, WSC_idx)

# Step 1c: create TX group
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid     # add group_id to each transactions
txn_2.group = gid

# Step 2: Bob signs txn_1,  Smart signature to authorizes txn_2
stxn_1 = txn_1.sign(Bob['private'])
encodedProg = Vending['result'].encode()              
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

# Step 3: assemble transaction group and send
signed_group =  [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Test 2: Bob overpays
* Bob sends 0.3 ALGOS
* He asks for 0.1 WSC
* This should only work once Exercise 2 has been implemented

In [ ]:
sp = algod_client.suggested_params()

# Step 1a: Prepare ALGO payment TXN from Bob to Veding
amt_1 = int(0.3*1e6)                                              # <------ overpayment
txn_1 = PaymentTxn(Bob['public'], sp, Vending['hash'],amt_1)

# Step 1b: Prepare WSC transfer from Vending to Bob
WSC_decimals = algod_client.asset_info(WSC_idx)['params']['decimals']
amt_2 = int(0.1 * 10**WSC_decimals )
txn_2 = AssetTransferTxn(Vending['hash'], sp, Bob['public'], amt_2, WSC_idx)

# Step 1c: create TX group
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid     # add group_id to each transactions
txn_2.group = gid

# Step 2: Bob signs txn_1,  Smart signature to authorizes txn_2
stxn_1 = txn_1.sign(Bob['private'])
encodedProg = Vending['result'].encode()              
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

# Step 3: assemble transaction group and send
signed_group =  [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Test 3: Money laundering
* Alice pays 0.2 ALGOS
* But Bob should get the 0.1 WSC
* This should NOT work

In [ ]:
sp = algod_client.suggested_params()

# Step 1a: Prepare ALGO payment TXN from Bob to Veding
amt_1 = int(0.2*1e6)                                              
txn_1 = PaymentTxn(Alice['public'], sp, Vending['hash'],amt_1)            # <------ Alice pays for Bob!

# Step 1b: Prepare WSC transfer from Vending to Bob
WSC_decimals = algod_client.asset_info(WSC_idx)['params']['decimals']
amt_2 = int(0.1 * 10**WSC_decimals )
txn_2 = AssetTransferTxn(Vending['hash'], sp, Bob['public'], amt_2, WSC_idx)

# Step 1c: create TX group
gid = transaction.calculate_group_id([txn_1, txn_2])
txn_1.group = gid     # add group_id to each transactions
txn_2.group = gid

# Step 2: Bob signs txn_1,  Smart signature to authorizes txn_2
stxn_1 = txn_1.sign(Bob['private'])
encodedProg = Vending['result'].encode()              
program = base64.decodebytes(encodedProg)
lsig = LogicSig(program)
stxn_2 = LogicSigTransaction(txn_2, lsig)

# Step 3: assemble transaction group and send
signed_group =  [stxn_1, stxn_2]
txid = algod_client.send_transactions(signed_group)

# Step 4: wait for confirmation
txinfo = wait_for_confirmation(algod_client, txid)

#### Get the status after the swap

In [ ]:
asset_holdings_df2(algod_client, MyAlgo['public'], Bob['public'], ["MyAlgo","Bob"])